In [47]:
import numpy as np
import random
from collections import deque
import pygame
import multiprocessing as mp
import time


In [48]:
# Функция активации ReLU
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

In [49]:
# Змейка
class Snake(object):
    def __init__(self, n):
        self.stack = deque()
        self.stack.append((n // 2, n // 2)) # позиция старта
        self.direction = (0, 1)  # начальное направление вправо
        self.size = n  # размер поля
        self.visited = set()  # посещенные клетки 

    # направление змейки
    def set_direction(self, direction):
        opposite_directions = {
            (0, 1): (0, -1),
            (0, -1): (0, 1),
            (1, 0): (-1, 0),
            (-1, 0): (1, 0),
        }
        if direction != opposite_directions.get(self.direction):
            self.direction = direction

    # добавление нового сегмента зменыша
    def add_snake(self):
        head_x, head_y = self.stack[0]
        new_head_x = head_x + self.direction[0]
        new_head_y = head_y + self.direction[1]
        self.stack.appendleft((new_head_x, new_head_y)) 
        self.visited.add((new_head_x, new_head_y))

    # движение змейки
    def move(self, food):
        self.add_snake()
        if self.stack[0] == food.position:
            food.new_food(self.stack)
        else:
            self.stack.pop()

    # получение головы
    def get_head_position(self):
        return self.stack[0]

    # получение всех сегментов змейки
    def get_snake_positions(self):
        return list(self.stack)

    # проверка на столкновение
    def has_collided(self):
        head = self.get_head_position()
        return (
            head[0] < 0
            or head[1] < 0
            or head[0] >= self.size
            or head[1] >= self.size
            or head in list(self.stack)[1:]
        )
    
    # проверка на свободу(как в сша)
    def is_cell_free(self, position):
        return (
            0 <= position[0] < self.size
            and 0 <= position[1] < self.size
            and position not in self.stack
        )

    # расстояние до еды(евклидово растояние)
    def distance_to_food(self, food_position):
        head_pos = self.get_head_position()
        return np.sqrt(
            (food_position[0] - head_pos[0]) ** 2
            + (food_position[1] - head_pos[1]) ** 2
        )

    # вычисляем угол между текущим направлением змейки и направлением на еду
    def angle_to_food(self, food_position):
        head_pos = self.get_head_position()
        direction_vector = [
            food_position[0] - head_pos[0],
            food_position[1] - head_pos[1],
        ]
        head_direction = list(self.direction)
        dot_product = np.dot(direction_vector, head_direction)
        magnitude_product = np.linalg.norm(direction_vector) * np.linalg.norm(
            head_direction
        )
        if magnitude_product == 0:
            return 0
        angle = np.arccos(dot_product / magnitude_product)
        return angle

    def calculate_sensors(self, food, grid_size):
        head_x, head_y = self.get_head_position()
        food_x, food_y = food.position
        sensors = []

        # Сенсоры для вертикальных, горизонтальных и диагональных направлений
        directions = [
            (-1, 0), (1, 0), (0, -1), (0, 1), 
            (-1, -1), (-1, 1), (1, -1), (1, 1)
        ]

        for dx, dy in directions:
            distance = 0
            sensor_value = 0
            temp_x, temp_y = head_x, head_y
            while True:
                temp_x += dx
                temp_y += dy
                distance += 1

                if not (0 <= temp_x < grid_size and 0 <= temp_y < grid_size):
                    sensor_value = 1 / distance
                    break

                if (temp_x, temp_y) in self.get_snake_positions():
                    sensor_value = 1 / distance
                    break

                if (temp_x, temp_y) == (food_x, food_y):
                    sensor_value = 2 / distance
                    break

                if temp_x in [0, grid_size-1] or temp_y in [0, grid_size-1]:
                    sensor_value = 1 / distance
                    break
            
            sensors.append(sensor_value)

        # Сенсоры для направления, где находится еда
        sensors.extend([
            int(food_y < head_y),
            int(food_y > head_y), 
            int(food_x < head_x), 
            int(food_x > head_x)   
        ])

        # Сенсоры близости к стенам для четырех основных направлений
        wall_distances = [
            head_y,              # Расстояние до верхней стены
            grid_size - head_y,  # Расстояние до нижней стены
            head_x-1,              # Расстояние до левой стены
            grid_size - head_x   # Расстояние до правой стены
        ]

        pos=self.get_snake_positions()
        dist_by_tel=10
        x_v=0
        x_y=0
        self.get_head_position()
        for i in range(len(pos)-1):
            if pos[0][0]!=pos[i][0] and pos[0][1]!=pos[i][1]:
               dist=np.sqrt((pos[i][0] - pos[0][0]) ** 2 + (pos[i][1] - pos[0][1]) ** 2)
               if dist <= dist_by_tel:
                   dist_by_tel=dist
                   x_v=pos[0][0]-pos[i][0]/abs(pos[0][0]-pos[i][0])
                   y_v=pos[0][1]-pos[i][1]/abs(pos[0][1]-pos[i][1])
                
        sensors.extend([
            int(x_v),
            int(x_v),
            int(dist_by_tel)
            ])

        # Нормализация расстояний путем деления на размер сетки
        normalized_wall_distances = [dist / grid_size for dist in wall_distances]
        sensors.extend(normalized_wall_distances)

        return sensors

    #возвращает все возможные направления движения
    def get_possible_directions(self):
        return [(0, 1), (0, -1), (1, 0), (-1, 0)]

    #оценка направления
    def evaluate_direction(self, direction, food):
        head_x, head_y = self.get_head_position()
        new_head_x = head_x + direction[0]
        new_head_y = head_y + direction[1]
        new_head_position = (new_head_x, new_head_y)
        
        if not self.is_cell_free(new_head_position):
            return -1  # Если позиция не свободна, это плохой выбор

        distance_to_food = np.sqrt((food.position[0] - new_head_x) ** 2 + (food.position[1] - new_head_y) ** 2)
        return -distance_to_food  # Чем ближе к еде, тем лучше



In [50]:
#еда
class Food(object):
    def __init__(self, n):
        self.size = n
        self.position = (random.randint(0, n - 1), random.randint(0, n - 1)) 

    #создание новой еды
    def new_food(self, snake_positions):
        while True:
            new_position = (random.randint(0, self.size - 1), random.randint(0, self.size - 1)) 
            if new_position not in snake_positions:
                self.position = new_position
                break


In [51]:
#нейронка
class NeuralNetwork(object):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        self.input_size = input_size
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.output_size = output_size
        self.weights1 = np.random.randn(self.input_size, self.hidden_size1)
        self.weights2 = np.random.randn(self.hidden_size1, self.hidden_size2)
        self.weights3 = np.random.randn(self.hidden_size2, self.output_size)

    def forward(self, x):
        self.hidden1 = relu(np.dot(x, self.weights1))
        self.hidden2 = relu(np.dot(self.hidden1, self.weights2))
        self.output = np.dot(self.hidden2, self.weights3)
        return self.output

In [52]:
# Функция для выполнения кроссовера двух родительских сетей
def crossover(parent1, parent2):
    child1_weights1 = np.copy(parent1.weights1)
    child2_weights1 = np.copy(parent2.weights1)
    child1_weights2 = np.copy(parent1.weights2)
    child2_weights2 = np.copy(parent2.weights2)
    child1_weights3 = np.copy(parent1.weights3)
    child2_weights3 = np.copy(parent2.weights3)

    crossover_point1 = np.random.randint(0, parent1.weights1.size)
    crossover_point2 = np.random.randint(0, parent1.weights2.size)
    crossover_point3 = np.random.randint(0, parent1.weights3.size)

    child1_weights1.flat[crossover_point1:], child2_weights1.flat[crossover_point1:] = (
        child2_weights1.flat[crossover_point1:],
        child1_weights1.flat[crossover_point1:],
    )
    child1_weights2.flat[crossover_point2:], child2_weights2.flat[crossover_point2:] = (
        child2_weights2.flat[crossover_point2:],
        child1_weights2.flat[crossover_point2:],
    )
    child1_weights3.flat[crossover_point3:], child2_weights3.flat[crossover_point3:] = (
        child2_weights3.flat[crossover_point3:],
        child1_weights3.flat[crossover_point3:],
    )

    child1 = NeuralNetwork(parent1.input_size, parent1.hidden_size1, parent1.hidden_size2, parent1.output_size)
    child2 = NeuralNetwork(parent1.input_size, parent1.hidden_size1, parent1.hidden_size2, parent1.output_size)
    child1.weights1 = child1_weights1
    child1.weights2 = child1_weights2
    child1.weights3 = child1_weights3
    child2.weights1 = child2_weights1
    child2.weights2 = child2_weights2
    child2.weights3 = child2_weights3

    return child1, child2


# Функция для мутации нейронной сети
def mutate(network, mutation_rate):
    if np.random.rand() < mutation_rate:
        mutation_point = np.random.randint(network.weights1.size)
        network.weights1.flat[mutation_point] += np.random.randn()
    if np.random.rand() < mutation_rate:
        mutation_point = np.random.randint(network.weights2.size)
        network.weights2.flat[mutation_point] += np.random.randn()
    if np.random.rand() < mutation_rate:
        mutation_point = np.random.randint(network.weights3.size)
        network.weights3.flat[mutation_point] += np.random.randn()
    return network


# Функция для оценки фитнеса
def fitness(snake, food, network,generation, max_steps=500, visualize=False):
    steps = 0
    score = 0
    visited_positions = set()
    eat=(generation//100 +1)
    crash=False
    while not crash and steps < max_steps:
        
        sensors = snake.calculate_sensors(food, snake.size)
        input_vector = np.array(sensors, dtype=float)
        output = network.forward(input_vector)
        direction = np.argmax(output)

        if direction == 0:
            snake.set_direction((0, 1))  
        elif direction == 1:
            snake.set_direction((0, -1))  
        elif direction == 2:
            snake.set_direction((1, 0))  
        elif direction == 3:
            snake.set_direction((-1, 0))  
        snake.move(food)
        steps += 1

        if visualize:
            visualize_game(snake, food)

        if snake.get_head_position() in visited_positions:
            score -= 1
        else:
            visited_positions.add(snake.get_head_position())

        # score += 1.0 / (snake.distance_to_food(food.position) + 10)
        crash=snake.has_collided()
        
    if crash:
        score-=5

    score += 3 * (len(snake.get_snake_positions()) - 1)
    print(score)
    return score

In [53]:
import numpy as np
import random
import matplotlib.pyplot as plt
import json
import pygame
import multiprocessing as mp

class GeneticAlgorithm:
    def __init__(
        self,
        population_size,
        mutation_rate,
        input_size,
        hidden_size1,
        hidden_size2,
        output_size,
        generations,
    ):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations
        self.population = [
            NeuralNetwork(input_size, hidden_size1, hidden_size2, output_size)
            for _ in range(population_size)
        ]
        self.fitness_scores_per_generation = []
        self.food_eaten_per_generation = []

    def evolve(self):
        for generation in range(self.generations):
            fitness_scores = []
            food_eaten = []

            for network in self.population:
                snake = Snake(10) 
                food = Food(10)   
                score = fitness(snake, food, network,generation, visualize=False)
                print(score)
                fitness_scores.append((score, network))
                food_eaten.append(score - len(snake.get_snake_positions()) + 1)

            self.fitness_scores_per_generation.append([fs[0] for fs in fitness_scores])
            self.food_eaten_per_generation.append(np.mean(food_eaten))

            fitness_scores.sort(key=lambda x: x[0], reverse=True)
            new_population = [
                network for _, network in fitness_scores[: self.population_size // 2]
            ]

            while len(new_population) < self.population_size:
                parent1, parent2 = random.sample(new_population, 2)
                child1, child2 = crossover(parent1, parent2)
                new_population.extend(
                    [
                        mutate(child1, self.mutation_rate),
                        mutate(child2, self.mutation_rate),
                    ]
                )

            self.population = new_population[: self.population_size]
            print(f"Поколение {generation + 1} | Лучшая приспособленность: {fitness_scores[0][0]}")

            best_network = self.population[0]
            best_snake = Snake(10)  
            best_food = Food(10)    
            fitness(best_snake, best_food, best_network,generation, visualize=True)

        self.save_stats_to_file("genetic_algorithm_stats.json")
        self.plot_stats()

    def print_stats(self):
        print("Статистика поколений:")
        for i, (fitness_scores, food_eaten) in enumerate(zip(self.fitness_scores_per_generation, self.food_eaten_per_generation)):
            print(f"Поколение {i + 1}:")
            print(f"  Максимальная приспособленность: {np.max(fitness_scores)}")
            print(f"  Средняя приспособленность: {np.mean(fitness_scores)}")
            print(f"  Среднее количество съеденной еды: {food_eaten}")

        self.print_food_eaten_stats()

    def print_food_eaten_stats(self):
        half = len(self.food_eaten_per_generation) // 2
        first_half = self.food_eaten_per_generation[:half]
        second_half = self.food_eaten_per_generation[half:]

        print(
            f"Среднее количество съеденной еды в первой половине обучения: {np.mean(first_half)}"
        )
        print(
            f"Среднее количество съеденной еды во второй половине обучения: {np.mean(second_half)}"
        )

    def save_stats_to_file(self, filename):
        data = {
            "fitness_scores_per_generation": self.fitness_scores_per_generation,
            "food_eaten_per_generation": self.food_eaten_per_generation,
        }
        with open(filename, "w") as f:
            json.dump(data, f)

    def plot_stats(self):
        generations = list(range(1, self.generations + 1))
        max_fitness = [np.max(scores) for scores in self.fitness_scores_per_generation]
        avg_fitness = [np.mean(scores) for scores in self.fitness_scores_per_generation]
        avg_food_eaten = self.food_eaten_per_generation

        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        plt.plot(generations, max_fitness, label="Максимальная приспособленность")
        plt.plot(generations, avg_fitness, label="Средняя приспособленность")
        plt.xlabel("Поколение")
        plt.ylabel("Приспособленность")
        plt.title("Приспособленность по поколениям")
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(generations, avg_food_eaten, label="Среднее количество съеденной еды", color="orange")
        plt.xlabel("Поколение")
        plt.ylabel("Среднее количество съеденной еды")
        plt.title("Среднее количество съеденной еды по поколениям")
        plt.legend()

        plt.tight_layout()
        plt.savefig("genetic_algorithm_stats.png")
        plt.show()

# Функция визуализации игры
def visualize_game(snake, food):
    pygame.init()
    screen_size = 500
    grid_size = screen_size // snake.size
    screen = pygame.display.set_mode((screen_size, screen_size))
    clock = pygame.time.Clock()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            return

    screen.fill((0, 0, 0))
    for pos in snake.get_snake_positions():
        pygame.draw.rect(
            screen,
            (0, 255, 0),
            (pos[1] * grid_size, pos[0] * grid_size, grid_size, grid_size),
        )
    pygame.draw.rect(
        screen,
        (255, 0, 0),
        (
            food.position[1] * grid_size,
            food.position[0] * grid_size,
            grid_size,
            grid_size,
        ),
    )

    pygame.display.flip()
    clock.tick(10)






In [ ]:
population_size = 100
mutation_rate = 0.05
input_size = 19  # 8 для направлений + 4 для сектора еды + 4 для близости к стенам + 3 к направлению хваста змеи(2) и растояние до хваста 
hidden_size1 = 16
hidden_size2 = 8
output_size = 4
generations = 50000

ga = GeneticAlgorithm(
    population_size, mutation_rate, input_size, hidden_size1, hidden_size2, output_size, generations
)
ga.evolve()

pygame 2.5.2 (SDL 2.28.3, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html
